# ペーパートレード（仮想取引）

このノートブックでは、PyJQuantsのペーパートレード機能を使って、リスクなしで取引戦略をテストする方法を学びます。

## ペーパートレードとは？

ペーパートレードは、実際のお金を使わずに株式取引をシミュレーションすることです。
- リスクなしで取引の練習ができる
- 投資戦略のテストに最適
- 実際の株価データを使用するので現実的

## 目次
1. 環境設定
2. トレーダーの作成
3. 注文の発注
4. 約定のシミュレーション
5. ポートフォリオの確認

## 1. 環境設定

In [ ]:
import os
from datetime import date
from decimal import Decimal

# 認証情報の設定
os.environ['JQUANTS_MAIL_ADDRESS'] = 'your_email@example.com'
os.environ['JQUANTS_PASSWORD'] = 'your_password'

import pyjquants as pjq
print('準備完了')

## 2. トレーダーの作成

まず、仮想の証券口座（トレーダー）を作成します。

In [ ]:
# 1,000万円の資金でトレーダーを作成
trader = pjq.Trader(initial_cash=10_000_000)

print(f'初期資金: {trader.cash:,.0f}円')
print(f'保有銘柄数: {len(trader.positions())}銘柄')

## 3. 注文の発注

### 成行注文（Market Order）

成行注文は、現在の市場価格で即座に約定する注文です。

In [ ]:
# トヨタ株を取得
toyota = pjq.Stock('7203')
print(f'{toyota.name}の現在値: {toyota.latest_price.close:,.0f}円')

# 成行で100株の買い注文を発注
order1 = trader.buy(toyota, quantity=100)

print(f'\n注文ID: {order1.id}')
print(f'銘柄: {order1.stock.code}')
print(f'売買: {order1.side.value}')
print(f'数量: {order1.quantity}株')
print(f'注文種別: {order1.order_type.value}')
print(f'状態: {order1.status.value}')

### 指値注文（Limit Order）

指値注文は、指定した価格以下（買い）または以上（売り）で約定する注文です。

In [ ]:
# ソニー株を取得
sony = pjq.Stock('6758')
current_price = sony.latest_price.close
print(f'{sony.name}の現在値: {current_price:,.0f}円')

# 現在値の5%下で指値買い注文
limit_price = float(current_price) * 0.95
order2 = trader.buy(sony, quantity=50, price=limit_price)

print(f'\n指値買い注文を発注')
print(f'指値: {order2.price:,.0f}円')
print(f'状態: {order2.status.value}')

### 未約定注文の確認

In [ ]:
# 未約定の注文一覧
pending = trader.pending_orders()

print(f'未約定注文: {len(pending)}件\n')
for order in pending:
    price_str = f'{order.price:,.0f}円' if order.price else '成行'
    print(f'  {order.stock.code}: {order.side.value} {order.quantity}株 @ {price_str}')

## 4. 約定のシミュレーション

過去の日付を指定して、その日の株価で注文を約定させます。

In [ ]:
# 最新の取引日で約定をシミュレーション
simulation_date = toyota.latest_price.date
print(f'シミュレーション日: {simulation_date}')

# 約定を実行
executions = trader.simulate_fills(simulation_date)

print(f'\n約定件数: {len(executions)}件\n')
for ex in executions:
    print(f'約定: {ex.stock.code} {ex.side.value} {ex.quantity}株 @ {ex.price:,.0f}円')
    print(f'      約定金額: {ex.value:,.0f}円')

## 5. ポートフォリオの確認

### 現金残高と保有銘柄

In [ ]:
# 現金残高
print(f'現金残高: {trader.cash:,.0f}円')
print(f'ポートフォリオ評価額: {trader.portfolio.total_value:,.0f}円')
print()

# 保有銘柄
positions = trader.positions()
print(f'保有銘柄数: {len(positions)}銘柄\n')

for pos in positions:
    print(f'{pos.stock.name} ({pos.stock.code})')
    print(f'  保有数: {pos.quantity}株')
    print(f'  平均取得単価: {pos.average_cost:,.0f}円')
    print(f'  評価額: {pos.market_value:,.0f}円')
    print(f'  含み損益: {pos.unrealized_pnl:,.0f}円 ({pos.unrealized_pnl_percent:+.2f}%)')
    print()

### ポートフォリオサマリー

In [ ]:
# サマリーをDataFrameで表示
summary = trader.portfolio.summary()
if len(summary) > 0:
    summary

### 売却してみる

In [ ]:
# トヨタ株を50株売却
if trader.position(toyota):
    sell_order = trader.sell(toyota, quantity=50)
    print(f'売り注文を発注: {sell_order}')
    
    # 約定
    executions = trader.simulate_fills(simulation_date)
    for ex in executions:
        print(f'約定: {ex.stock.code} {ex.side.value} {ex.quantity}株 @ {ex.price:,.0f}円')
    
    print(f'\n実現損益: {trader.portfolio.realized_pnl:,.0f}円')
else:
    print('トヨタ株を保有していません')

### 取引履歴

In [ ]:
# 取引履歴を表示
history = trader.portfolio.history()
if len(history) > 0:
    print('取引履歴:')
    history

## 注文のキャンセル

In [ ]:
# 未約定の指値注文があればキャンセル
pending = trader.pending_orders()
if pending:
    for order in pending:
        if trader.cancel_order(order):
            print(f'注文をキャンセルしました: {order.stock.code}')
else:
    print('未約定の注文はありません')

## まとめ

このノートブックでは以下を学びました：

- **Trader**: 仮想の証券口座を作成
- **buy() / sell()**: 成行・指値注文の発注
- **simulate_fills()**: 過去の株価で約定をシミュレーション
- **Portfolio**: 保有銘柄と損益の確認

次のノートブック「04_ポートフォリオ分析.ipynb」では、より高度なポートフォリオ分析を行います。